In [1]:
from bs4 import BeautifulSoup
from urllib import request
from time import sleep

import pandas as pd

IOP_S = "https://iopscience.iop.org/nsearch?terms="
IOP_E = "&nextPage=2&previousPage=-1&currentPage="
IOP_P = "&searchDatePeriod=anytime&orderBy=relevance&pageLength=50"

def souparticlesIOP(material, page, max_int = 7, dream = 2):
    soup1 = None
    ints=0
    while ints<max_int:
        try:
            resp = request.urlopen(IOP_S+material+IOP_E+str(page)+IOP_P)
            soup1 = BeautifulSoup(resp, 'lxml')
            break
        except:
            print("Reintentando pag: " +str(page))
            sleep(dream)
            pass
        finally:
            ints +=1
    if(soup1==None):
        print("Por favor reinicie la busqueda para pag: " + str(page))
        return "Por favor reinicie la busqueda";
    else:
        print("Añadida pag: " + str(page))
        return soup1.find_all('div', {"class" : "art-list-item-body"})

def find_all(material):
    all_     = []
    to_retry = []
    for i in range(1,12):
        item = souparticlesIOP(material, i)
        if(isinstance(item, str)):
            to_retry.append(i)
        else:
            all_.append(item)
    return all_;

In [13]:

def stringify(raw_pages):
    arts_db = []
    for page in raw_pages:
        for element in page:
            try:
                title     =     element.findChild('h2' , {"class"   : "art-list-item-title"}).findChild("a")          # 0 title
                abstract  =     element.findChild('div', {"class"   : "article-text view-text-small"}).findChild("p") # 1 abstract
                DOI       =     element.findChild('a'  , {"class"   : "mr-2"})                                        # 2 DOI
                journal   =     element.findChild('em')                                                               # 3 Journal
                vol       =     element.findChild('b')                                                                # 4 vol
                authors   =     element.find_all('span', {"itemprop": "author"})                                      # 5 auths *beta*

                yearfind  =     element.find_all("p"   , { "class" :"small art-list-item-meta"})                      # 6 year
                year      =     int(yearfind[1].text[14:19])

                arts_db.append([title.text ,abstract.text, DOI.text, journal.text, vol.text ,[auth.getText() for auth in authors], year])
            except:
                pass

    return arts_db;

In [14]:
def panda_to_me(all_arts):
    test_arts = {}
    for i in range(0,7):
        ls = []
        for j in range(0,len(all_arts)):
            ls.append(all_arts[j][i])
        if i == 0:
            test_arts.update(Title        = ls)
        elif i == 1:
            test_arts.update(Abstract     = ls)
        elif i == 2:
            test_arts.update(DOI          = ls)
        elif i == 3:
            test_arts.update(Journal      = ls)
        elif i == 4:
            test_arts.update(Vol          = ls)
        elif i == 5:
            test_arts.update(Auths        = ls)
        else:
            test_arts.update(Year         = ls)
    return pd.DataFrame(test_arts);

In [4]:
ZnTe = find_all("ZnTe")

Reintentando pag: 1
Añadida pag: 1
Reintentando pag: 2
Reintentando pag: 2
Reintentando pag: 2
Reintentando pag: 2
Reintentando pag: 2
Añadida pag: 2
Añadida pag: 3
Añadida pag: 4
Reintentando pag: 5
Añadida pag: 5
Reintentando pag: 6
Reintentando pag: 6
Añadida pag: 6
Reintentando pag: 7
Reintentando pag: 7
Reintentando pag: 7
Reintentando pag: 7
Reintentando pag: 7
Reintentando pag: 7
Reintentando pag: 7
Por favor reinicie la busqueda para pag: 7
Reintentando pag: 8
Reintentando pag: 8
Reintentando pag: 8
Añadida pag: 8
Reintentando pag: 9
Reintentando pag: 9
Reintentando pag: 9
Reintentando pag: 9
Reintentando pag: 9
Reintentando pag: 9
Reintentando pag: 9
Por favor reinicie la busqueda para pag: 9
Añadida pag: 10
Reintentando pag: 11
Reintentando pag: 11
Reintentando pag: 11
Añadida pag: 11


In [15]:
ZnTe_db    = stringify(ZnTe)

In [16]:
ZnTe_panda = panda_to_me(ZnTe_db)

In [19]:
ZnTe_panda["Year"].unique()

array([1980, 2019, 2018, 2017, 2006, 2016, 2014, 2015, 1967, 2010, 2013,
       1996, 1994, 2009, 1983, 2011, 1969, 2004, 1975, 1963, 1998, 1993,
       2002, 1970, 1979, 1977, 2003, 1999, 2000, 1978, 1984, 2012, 2007,
       1973, 2008, 2001, 1985, 1972, 1986, 1995, 1971, 1990, 1988, 2005,
       2020, 1966, 1997, 1989, 1976, 1992, 1982, 1991, 1987, 1965, 1968,
       1981, 1964], dtype=int64)